# Ejercicio

In [33]:
import pandas as pd
import os
import spacy

nlp = spacy.load('es_core_news_lg')

### Lectura de archivos

In [34]:
#Obtengo los nombres de los archivos csv
files = os.listdir("./corpus")

In [35]:
data = []
for file in files:
    with open("./corpus/" + file, "r", encoding="utf-8") as f:
        text = f.read()

        #Elimino los saltos de linea
        text = text.replace("\n", " ")

        #Agrego el archivo al conjunto de diccionarios
        data.append({"filename": file, "text": text})

#Creo el dataframe
df = pd.DataFrame(data)

### Limpieza de datos

In [36]:
#Palabras que no aportan informacion
ruido = ["decir","llamar","hola","pregunta","buenas","buenos","dias","dia","tardes","tarde","noches","gracias","adios","porfavor","disculpe","favor","favor","nombre","bienvenido","bienvenida"]

def es_token_relevante(token):
    return not token.is_stop \
        and not token.is_punct \
        and not token.like_num \
        and not token.is_space \
        and (token.text not in ruido)

def procesar_tokens(texto):
    texto = texto.lower()

    #Obtengo los tokens
    doc = nlp(texto)

    #Filtro y lematizo los tokens
    tokens_relevantes = [token.lemma_ for token in doc if es_token_relevante(token)]

    return tokens_relevantes

df["corpus"] = df["text"].apply(procesar_tokens)

In [37]:
#Filtro los corpus que no tienen tokens
df[df['corpus'].apply(len) == 0]

,filename,text,corpus
258,34774-17-54-37.csv,Pregunta Treinta y ocho,[]
265,34780-09-04-00.csv,Pregunta también .,[]
326,34783-11-34-40.csv,"Pregunta Es el de dieciséis , siete , tres , ...",[]
442,34794-16-58-19.csv,"Pregunta . , ya . Y",[]
599,34806-12-20-06.csv,"Pregunta , no .",[]
609,34806-18-35-33.csv,"Pregunta , ya , Sí , . , , sí , sí , . Ya y ....",[]


Después de limpiar los corpus, hay conversaciones que quedaron sin tokens. Eso es porque las únicas palabras en el llamado fueron stop words o ruido, es decir que se usaron palabras que no aportaron contexto a la conversación. Separo los corpus que tienen tópico de los que no y los etiqueto como 'Sin tópico'.

In [38]:
#Separo los corpus que tienen tokens de los que no
df_sin_topico = df[df['corpus'].apply(len) == 0]
df = df[df['corpus'].apply(len) > 0]

#Le asigno 'Sin topico' a los corpus que no tienen tokens
df_sin_topico['topico'] = 'Sin topico'

#Elimino la columna corpus
df_sin_topico.drop(columns=['corpus'], inplace=True)

### Embeddings

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_embedding(corpus):
    
    #Instancio el vectorizador adentro de la funcion para que no se guarde el vocabulario
    vectorizer = TfidfVectorizer(stop_words=None)
    embedding = vectorizer.fit_transform(corpus)
    tokens = vectorizer.get_feature_names_out()

    #Devuelvo una serie con el embedding y los tokens para poder asignarlos a las columnas del dataframe
    return pd.Series([embedding, tokens], index=['embedding', 'corpus'])

#Creo los embeddings del corpus
df[['embedding', 'corpus']] = df['corpus'].apply(get_embedding)

### Extracción de tópicos

In [40]:
from sklearn.decomposition import NMF
from gensim.models import LdaModel, HdpModel
from gensim.corpora.dictionary import Dictionary

In [41]:
def get_topics(row, n_topics=3):

    #Obtengo el embedding y los tokens de la fila
    embedding = row['embedding']
    tokens = row['corpus']

    #Instancio el modelo NMF y lo entreno
    nmf = NMF(n_components=n_topics, random_state=42)
    nmf.fit(embedding)

    #Obtengo los tópicos
    top_words = nmf.components_.argsort()[:, -5:][0]
    topicos = [tokens[i] for i in top_words]
    
    return topicos

df['topico_nmf'] = df.apply(lambda x: get_topics(x, n_topics=3), axis=1)

C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\decomposition\_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\decomposition\_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\decomposition\_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-

In [42]:
def get_topic(corpus):

    #Creo un bolsa de palabras e inicializo el diccionario
    dictionary = Dictionary([corpus])
    bow = [dictionary.doc2bow(doc) for doc in [corpus]]

    #Creo el modelo y extraigo los tópicos
    model = HdpModel(corpus=bow, id2word=dictionary, random_state= 30)
    topicos = model.print_topics()[0][1]

    #Eliminos los coeficientes para dejar los tópicos limpios
    topicos = topicos.replace('*', '+').replace('.','').split('+')
    topicos = [topico.strip() for topico in topicos if not topico.strip().isnumeric()]

    return topicos

df['topico_hdp'] = df['corpus'].apply(get_topic)

In [43]:
def get_topic(corpus):

    #Creo un bolsa de palabras e inicializo el diccionario
    dictionary = Dictionary([corpus])
    bow = [dictionary.doc2bow(doc) for doc in [corpus]]

    #Creo el modelo y extraigo los tópicos
    model = LdaModel(corpus=bow, id2word=dictionary, random_state= 30, num_topics=5)
    topicos = model.print_topics()[0][1]

    #Eliminos los coeficientes para dejar los tópicos limpios
    topicos = topicos.replace('*', '+').replace('.','').split('+')
    topicos = [topico.strip() for topico in topicos if not topico.strip().isnumeric()]

    return topicos

df['topico_lda'] = df['corpus'].apply(get_topic)

In [44]:
df

,filename,text,corpus,embedding,topico_nmf,topico_hdp,topico_lda
0,34701-10-07-17.csv,"Pregunta Hola , mi nombre es Valeria . Discul...","[abrir, agradecer, arroba, asegurar, cadena, c...","(0, 73)\t1.0\n (1, 29)\t1.0\n (2, 39)\t1.0...","[tía, josé, pedido, salmón, venir]","[necesitar, mirar, arroba, uva, agradecer, gua...","[""rehacer"", ""agradecer"", ""dejar"", ""constanteme..."
1,34701-12-31-48.csv,Pregunta Buenas tardes . punto Mi nombre es J...,"[absolutamente, aparecer, asignar, atención, a...","(0, 35)\t1.0\n (1, 21)\t1.0\n (2, 17)\t1.0...","[reservado, preocupar, ayudar, gutiérrez, pedido]","[recién, pedido, josé, indíqueme, ayudar, preo...","[""meter"", ""buscar"", ""oir"", ""generar"", ""correo""..."
2,34701-12-57-22.csv,Pregunta Hola . Buenas tardes . Habla con Mar...,"[agradecer, azcárate, consulta, demorar, despa...","(0, 6)\t1.0\n (1, 1)\t1.0\n (2, 2)\t1.0\n ...","[maría, voisin, agradecer, mon, demorar]","[despacho, demorar, maría, agradecer, consulta...","[""jueves"", ""azcárate"", ""maría"", ""demorar"", ""mo..."
3,34701-13-27-47.csv,"Pregunta Buenas tardes , Francisco Moreno . ,...","[ahorrar, comunicar, correcto, desaparecer, em...","(0, 5)\t1.0\n (1, 14)\t1.0\n (2, 13)\t1.0\...","[llegar, yo, perder, correcto, pedido]","[página, francisco, eme, jota, mirar, comunica...","[""ir"", ""pasar"", ""comunicar"", ""francisco"", ""des..."
4,34701-13-56-30.csv,"Pregunta al final . Hola , buenas tardes con ...","[ana, aparecer, arepa, bolsa, comprar, corrobo...","(0, 11)\t1.0\n (1, 14)\t1.0\n (2, 24)\t1.0...","[precio, máquina, oferta, página, aparecer]","[retiro, mirar, bolsa, mandar, querer, aparece...","[""dirección"", ""máquina"", ""retiro"", ""aparecer"",..."
...,...,...,...,...,...,...,...
635,34809-12-21-48.csv,Pregunta barba . Buenas tardes . . tengo una ...,"[atender, ayer, barba, cambiar, cancelar, celu...","(0, 2)\t1.0\n (1, 6)\t1.0\n (2, 20)\t1.0\n...","[pedido, página, salir, decir, llegar]","[pedido, corresponder, venir, ver, consulta, m...","[""sustituir"", ""llegar"", ""celular"", ""mandar"", ""..."
636,34809-12-37-51.csv,Pregunta Hola . Buenos días . Mi nombre es Pa...,"[abrir, acabar, aceptar, amable, bonito, cajer...","(0, 36)\t1.0\n (1, 46)\t1.0\n (2, 12)\t1.0...","[perdón, amable, confirmar, despacho, querer]","[error, duplicar, promoción, compra, proceso, ...","[""despacho"", ""información"", ""cajera"", ""ingresa..."
637,34809-12-42-09.csv,Pregunta Buenas tardes . Bienvenido . . mi no...,"[anotar, anulación, anulado, anular, aparecer,...","(0, 38)\t1.0\n (1, 56)\t1.0\n (2, 57)\t1.0...","[tarjeta, sistema, perfecto, entrega, pedido]","[solicitar, inmediatamente, lunes, boleta, mai...","[""problema"", ""par"", ""opción"", ""reclamo"", ""rela..."
638,34809-12-45-28.csv,"Pregunta . Mira , tengo una pura . Es una com...","[abrir, acceso, actualizar, alguien, alternati...","(0, 52)\t1.0\n (1, 77)\t1.0\n (2, 18)\t1.0...","[unidad, exactamente, mencionar, compra, produ...","[mastercard, llamada, acceso, plástico, abrir,...","[""solitar"", ""incidencia"", ""oficial"", ""pedir"", ..."


### Evaluación

No hay una forma muy directa para comparar el desempeño de estos 3 algoritmos porque usan 3 encares distintos. Non-negative Matrix Fatorization (NMF) usa algebra lineal para la descomposición de tópicos, Hierarchical Dirichlet Process (HDP) busca agrupar un conjunto de datos según su distribución, y Latent Dirichlet Allocation (LDA) calcula las probabilidades de que una palabra pertenezca a un conjunto. A su vez son 640 casos distintos (modelos distintos, no relacionados), lo cual dificulta  un poco el análisis.

Luego de evaluar los distintos modelos, viendo que conjunto de palabras representaban mejor a la conversación, decidí que el mejor modelo fue el Hierarchical Dirichlet Process.

In [45]:
#Elimino las columnas que no voy a usar más
df.drop(columns=['embedding', 'corpus','topico_nmf','topico_lda'], inplace=True)

#Renombro la columna de tópicos
df.rename(columns={'topico_hdp':'topico'}, inplace=True)

#Concateno los corpus que no tienen tópico con los que si
df = pd.concat([df, df_sin_topico])

In [47]:
df

,filename,text,topico
0,34701-10-07-17.csv,"Pregunta Hola , mi nombre es Valeria . Discul...","[necesitar, mirar, arroba, uva, agradecer, gua..."
1,34701-12-31-48.csv,Pregunta Buenas tardes . punto Mi nombre es J...,"[recién, pedido, josé, indíqueme, ayudar, preo..."
2,34701-12-57-22.csv,Pregunta Hola . Buenas tardes . Habla con Mar...,"[despacho, demorar, maría, agradecer, consulta..."
3,34701-13-27-47.csv,"Pregunta Buenas tardes , Francisco Moreno . ,...","[página, francisco, eme, jota, mirar, comunica..."
4,34701-13-56-30.csv,"Pregunta al final . Hola , buenas tardes con ...","[retiro, mirar, bolsa, mandar, querer, aparece..."
...,...,...,...
635,34809-12-21-48.csv,Pregunta barba . Buenas tardes . . tengo una ...,"[pedido, corresponder, venir, ver, consulta, m..."
636,34809-12-37-51.csv,Pregunta Hola . Buenos días . Mi nombre es Pa...,"[error, duplicar, promoción, compra, proceso, ..."
637,34809-12-42-09.csv,Pregunta Buenas tardes . Bienvenido . . mi no...,"[solicitar, inmediatamente, lunes, boleta, mai..."
638,34809-12-45-28.csv,"Pregunta . Mira , tengo una pura . Es una com...","[mastercard, llamada, acceso, plástico, abrir,..."
